In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import time

###**Data Reading**

In [0]:
df= spark.read.format("parquet")\
.load("abfss://bronze@avidatabricksete.dfs.core.windows.net/products")

In [0]:
df = df.drop("_rescued_data")

In [0]:
df.display()

### **Functions**

In [0]:
df.createOrReplaceTempView("products")

In [0]:
%sql
CREATE OR REPLACE FUNCTION databrciks_catalog.bronze.discount_func(p_price double)
RETURNS double 
LANGUAGE SQL
RETURN p_price * 0.90


In [0]:
%sql
select product_id,price,databrciks_catalog.bronze.discount_func(price) as discounted_price from products

### **Use of Function in Pyspark**

In [0]:
df = df.withColumn("discounted_price",expr("databrciks_catalog.bronze.discount_func(price)"))
df.display()

In [0]:
%sql
CREATE OR REPLACE FUNCTION databrciks_catalog.bronze.upper_func(p_brand string)
RETURNS string 
LANGUAGE PYTHON
AS 
$$ 
  return p_brand.upper()
$$


In [0]:
%sql
select product_id,brand,databrciks_catalog.bronze.upper_func(brand) as brand_upper from products

In [0]:
df.write.format("delta")\
    .mode("overwrite")\
        .option("path","abfss://silver@avidatabricksete.dfs.core.windows.net/products")\
            .save()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databrciks_catalog.silver.products_silver
USING DELTA
LOCATION 'abfss://silver@avidatabricksete.dfs.core.windows.net/products'


In [0]:
%sql
SELECT * FROM databrciks_catalog.silver.products_silver
